In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [13]:
k_data = json.load(open("./KOK-VQA.json"))
k_df = pd.DataFrame(k_data["annotations"])

In [14]:
k_df["questions"]

0    [{'question_id': '1-1', 'question_ko': '사진속 어떤...
1    [{'question_id': '2-1', 'question_ko': '사진속 어떤...
2    [{'question_id': '1-1', 'question_ko': '이미지의 어...
Name: questions, dtype: object

In [15]:
k_df

,ACTION,CAPTION,HEIGHT,IMAGE_ACQUISITION_DATE,IMAGE_ACQUISITION_DATE,IMAGE_ID,IMAGE_NAME,IMAGE_URL,LICENSE,MAINOBJECT,MAINOBJECT_URL,SCENE,SUBOBJECT_1,SUBOBJECT_2,WIDTH,WIKI_FILE,bounding_box,questions
0,,,533,20181004.0,NaN,1,,https://upload.wikimedia.org/wikipedia/common...,FREE,piano,http://ko.dbpedia.org/resource/피아노,,Hands,Leg,800,Hupfeld piano played (2018-10-04 @pxhere 15460...,"[{'BOX_ID': '1-1', 'OBJECT': 'piano', 'X_COORD...","[{'question_id': '1-1', 'question_ko': '사진속 어떤..."
1,,,533,NaN,20181004,2,,https://upload.wikimedia.org/wikipedia/common...,FREE,pinpong,http://ko.dbpedia.org/resource/탁구,,person,table,800,Hupfeld piano played (2018-10-04 @pxhere 1546...,"[{'BOX_ID': '2-1', 'OBJECT': 'pingpong', 'X_CO...","[{'question_id': '2-1', 'question_ko': '사진속 어떤..."
2,,,533,20181004.0,NaN,3,,https://upload.wikimedia.org/wikipedia/common...,FREE,piano,http://ko.dbpedia.org/resource/피아노,,Hands,Leg,800,Hupfeld piano played (2018-10-04 @pxhere 15460...,"[{'BOX_ID': '1-1', 'OBJECT': 'piano', 'X_COORD...","[{'question_id': '1-1', 'question_ko': '이미지의 어..."


In [16]:
k_kb = json.load(open("./KOK-VQA-KB.json"))
kb_df = pd.DataFrame(k_kb["knowledgebase"])

In [17]:
kb_df.T

,KB,context,dataset,e1,e1_label,e2,e2_label,r,sources,surface_en,surface_ko
ko.dbpedia/1137,ko.dbpedia,None,None,http://ko.dbpedia.org/resource/피아노,피아노,피아노,피아노,http://dbpedia.org/ontology/title,None,None,"[피아노, 이름, 피아노]"
ko.dbpedia/1142,ko.dbpedia,None,None,http://ko.dbpedia.org/resource/피아노,피아노,None,바르톨로메오,http://dbpedia.org/ontology/abstract,None,None,"[피아노, 발명가, 크리스토포리]"
ko.dbpedia/1437,ko.dbpedia,None,None,http://ko.dbpedia.org/resource/탁구,탁구,http://ko.dbpedia.org/resource/탁구라켓,탁구라켓,http://dbpedia.org/ontology/equipment,None,None,"[탁구, 장비, 탁구라켓]"
conceptnet/1,conceptnet,None,/d/conceptnet/4/en,/c/en/piano/r,piano,/c/en/very_popular_musical_instrument,very popular musical instrument,/r/IsA,None,None,"[피아노, 이다, 매우 인기있는 악기]"
conceptnet/2,conceptnet,None,/d/conceptnet/4/en,/c/en/piano/r,piano,/c/en/music_store,music store,/r/AtLocation,None,None,"[피아노, 위치하다, 음반 가게]"


## ConceptNet

In [32]:
colnames = ["relation", "subject", "object", "license", "source", "process", "weight"]
df_co = pd.read_csv("../../data/conceptnet/conceptnet-assertions-5.7.0.csv", warn_bad_lines=False, error_bad_lines=False, names=colnames, header=None, low_memory=False)

/home/aailab_conda/anaconda3/envs/kbvqa/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
df_co[df_co["relation"] == "/a/[/r/RelatedTo/"]

,relation,subject,object,license,source,process,weight
17716847,/a/[/r/RelatedTo/,/c/aa/a/,/c/en/this/]\t/r/RelatedTo\t/c/aa/a\t/c/en/thi...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716848,/a/[/r/RelatedTo/,/c/aa/a/a/,/c/fr/ce/]\t/r/RelatedTo\t/c/aa/a/a\t/c/fr/ce\...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716849,/a/[/r/RelatedTo/,/c/aa/a_saku/r/,/c/fr/aujourd’hui/]\t/r/RelatedTo\t/c/aa/a_sak...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716850,/a/[/r/RelatedTo/,/c/aa/abal/n/,/c/fr/sang/]\t/r/RelatedTo\t/c/aa/abal/n\t/c/f...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716851,/a/[/r/RelatedTo/,/c/aa/abar/n/,/c/fr/sècheresse/]\t/r/RelatedTo\t/c/aa/abar/n...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716852,/a/[/r/RelatedTo/,/c/aa/abatasa/n/,/c/fr/alphabet/]\t/r/RelatedTo\t/c/aa/abatasa/...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716853,/a/[/r/RelatedTo/,/c/aa/abba/n/,/c/fr/chef_de_famille/]\t/r/RelatedTo\t/c/aa/a...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716854,/a/[/r/RelatedTo/,/c/aa/abba/n/,/c/fr/géniteur/]\t/r/RelatedTo\t/c/aa/abba/n\t...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716855,/a/[/r/RelatedTo/,/c/aa/abba/n/,/c/fr/président/]\t/r/RelatedTo\t/c/aa/abba/n\...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
17716856,/a/[/r/RelatedTo/,/c/aa/abba/n/,/c/fr/père/]\t/r/RelatedTo\t/c/aa/abba/n\t/c/f...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"


In [48]:
df_co[df_co["subject"] == "/c/en/piano/r/"]

,relation,subject,object,license,source,process,weight
16193396,/a/[/r/HasContext/,/c/en/piano/r/,/c/en/music/]\t/r/HasContext\t/c/en/piano/r\t/...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
19941017,/a/[/r/RelatedTo/,/c/en/piano/r/,/c/de/leise/]\t/r/RelatedTo\t/c/en/piano/r\t/c...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
19941018,/a/[/r/RelatedTo/,/c/en/piano/r/,/c/en/direction/]\t/r/RelatedTo\t/c/en/piano/r...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
19941019,/a/[/r/RelatedTo/,/c/en/piano/r/,/c/en/mezzo_piano/]\t/r/RelatedTo\t/c/en/piano...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
19941020,/a/[/r/RelatedTo/,/c/en/piano/r/,/c/en/mp/]\t/r/RelatedTo\t/c/en/piano/r\t/c/en...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
19941021,/a/[/r/RelatedTo/,/c/en/piano/r/,/c/en/pianissimo/]\t/r/RelatedTo\t/c/en/piano/...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
19941022,/a/[/r/RelatedTo/,/c/en/piano/r/,/c/en/pp/]\t/r/RelatedTo\t/c/en/piano/r\t/c/en...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
19941023,/a/[/r/RelatedTo/,/c/en/piano/r/,/c/en/sheet_music/]\t/r/RelatedTo\t/c/en/piano...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"
19941024,/a/[/r/RelatedTo/,/c/en/piano/r/,/c/en/softly/]\t/r/RelatedTo\t/c/en/piano/r\t/...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 1.0}"


In [60]:
df_co[df_co["subject"] == "/c/en/piano/"]["object"].iloc[37]

'/c/en/instrument_of_music/]\t/r/IsA\t/c/en/piano\t/c/en/instrument_of_music\t{"dataset": "/d/conceptnet/4/en"'

In [61]:
df_co[df_co["subject"] == "/c/en/piano/"]

,relation,subject,object,license,source,process,weight
84765,/a/[/r/AtLocation/,/c/en/piano/,/c/en/church/]\t/r/AtLocation\t/c/en/piano\t/c...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/rcarey1""}","{""activity"": ""/s/activity/omcs/vote"""
84766,/a/[/r/AtLocation/,/c/en/piano/,/c/en/concert/]\t/r/AtLocation\t/c/en/piano\t/...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/lennonkbc""}","{""activity"": ""/s/activity/omcs/omcs1_possibly..."
84767,/a/[/r/AtLocation/,/c/en/piano/,/c/en/concert_hall/]\t/r/AtLocation\t/c/en/pia...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/nihilmatt...","{""activity"": ""/s/activity/omcs/omcs1_possibly..."
84768,/a/[/r/AtLocation/,/c/en/piano/,/c/en/drawing_room/]\t/r/AtLocation\t/c/en/pia...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/rod57""}]","""surfaceEnd"": ""the drawing room"""
84769,/a/[/r/AtLocation/,/c/en/piano/,/c/en/music_room/]\t/r/AtLocation\t/c/en/piano...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/meowmille...","""surfaceEnd"": ""the music room"""
84770,/a/[/r/AtLocation/,/c/en/piano/,/c/en/music_school/]\t/r/AtLocation\t/c/en/pia...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/co...","""contributor"": ""/s/contributor/omcs/maofnoah""}]","""surfaceEnd"": ""...music school"""
84771,/a/[/r/AtLocation/,/c/en/piano/,/c/en/music_store/]\t/r/AtLocation\t/c/en/pian...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/bedume""}]","""surfaceEnd"": ""a music store"""
84772,/a/[/r/AtLocation/,/c/en/piano/,/c/en/neighbor's_house/]\t/r/AtLocation\t/c/en...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/annedog""}]","""surfaceEnd"": ""a neighbor's house"""
84773,/a/[/r/AtLocation/,/c/en/piano/,/c/en/old_movie/]\t/r/AtLocation\t/c/en/piano\...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/annedog""}]","""surfaceEnd"": ""an old movie"""
84774,/a/[/r/AtLocation/,/c/en/piano/,/c/en/orchestra/]\t/r/AtLocation\t/c/en/piano\...,"""license"": ""cc:by/4.0""","""sources"": [{""activity"": ""/s/activity/omcs/om...","""contributor"": ""/s/contributor/omcs/bjb""}","{""activity"": ""/s/activity/omcs/vote"""


In [52]:
df_co[df_co["subject"] == "/c/ko/피아노/"] 

,relation,subject,object,license,source,process,weight
2155975,/a/[/r/EtymologicallyRelatedTo/,/c/ko/피아노/,/c/en/piano/]\t/r/EtymologicallyRelatedTo\t/c/...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 0.25}"
8731825,/a/[/r/ExternalURL/,/c/ko/피아노/,/http://en.wiktionary.org/wiki/피아노/]\t/r/Exter...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 0.25}"
8731826,/a/[/r/ExternalURL/,/c/ko/피아노/,/http://fr.wiktionary.org/wiki/피아노/]\t/r/Exter...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/wikt...","""process"": ""/s/process/wikiparsec/2""}]","""weight"": 0.25}"
8731827,/a/[/r/ExternalURL/,/c/ko/피아노/,/http://ko.dbpedia.org/resource/피아노/]\t/r/Exte...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/dbpe...","""weight"": 1.0}",NaN
32496008,/a/[/r/Synonym/,/c/ko/피아노/,/c/en/piano/]\t/r/Synonym\t/c/ko/피아노\t/c/en/pi...,"""license"": ""cc:by-sa/4.0""","""sources"": [{""contributor"": ""/s/resource/dbpe...","""weight"": 0.5}",NaN


In [53]:
df_co[df_co["subject"] == "/c/ko/피아노/r"]

,relation,subject,object,license,source,process,weight
